# Monte Carlo Tallies

This notebook demonstrates three different tally types to compute the absorption rate in a 1D slab problem.  The three approaches are: 1) analog tally: only when an actual absorption collision takes place, 2) collision tally: at each collision we compute the flux and multiply by the absorption macro xs, 3) pathlength tally: at each flight we compute the flux and multiply by the absorption macro xs.

In [22]:
import numpy as np
import random

This is a monoenergetic problem with only scattering and absorption.  We are solving a 10cm slab and will tally absorption rates over a 1 cm mesh.

In [23]:
Sigma_t = 1.5    #0.5
Sigma_s = 1.2    #0.4
Sigma_a = Sigma_t - Sigma_s
A=12

L = 10
nps = 10000

This block sets the seed and initializes the tallies (mean) and the standard deviations.

In [24]:
random.seed(a=2)
analog_tally = np.zeros(L)
collision_tally = np.zeros(L)
track_tally = np.zeros(L)

analog_tally_uq = np.zeros(L)
collision_tally_uq = np.zeros(L)
track_tally_uq = np.zeros(L)

This block is the main loop.  In 1D, there is no need to track all three components of the direction vector, we only need the "w" component if we assume the slabs are along the "z" direction.  The is only one material in this problem, so the code tracks particles across the full length "L".  The tallies are done over a superimposed mesh that doesn't impact the tracking.

In [25]:
for i in range(nps):
    s_track = np.zeros(L)   #initialize the history score
    s_collision = np.zeros(L)
    s_analog = np.zeros(L)
    old_x = random.random()*L     #uniform source in space
    old_mu = random.random()*2 -1   #uniform angle in the LAB
    w=1.0   #initial particle weight
    while True:    #while the particle is alive
        s = -np.log(random.random())/Sigma_t    #sample distance traveled
        new_x = old_x + s*old_mu    #calculate new position 
        #track tally
        ind1 = int(old_x)        #mesh index of starting location
        ind2 = int(new_x)        #mesh index of new location
        ind2 = min(10,ind2)      #check the limits of the new location
        ind2 = max(0,ind2)
        if ind1 == ind2:         #if particle stays in same mesh
            s_track[ind1] = s_track[ind1] + w*s*Sigma_a     #add pathlength contribution to pathlength history score
        else:         
            if old_mu < 0:       #if the particle is moving left
                delta_x = old_x - float(ind1)
                s_track[ind1] = s_track[ind1] + abs(w*delta_x/old_mu)*Sigma_a   #contribution to the initial cell
                ind1 = ind1 - 1
                while (ind1 != ind2):     #contribution to all cells between starting and end point
                    s_track[ind1] = s_track[ind1] + abs(w*1.0/old_mu)*Sigma_a
                    ind1 = ind1 - 1
                delta_x = float(ind1+1) - new_x
                if (new_x > 0.0 and new_x < 10.0):   #contribution to end cell
                    s_track[ind1] = s_track[ind1] + abs(w*delta_x/old_mu)*Sigma_a
            else:               #if particle is moving right
                delta_x = float(ind1+1) - old_x
                s_track[ind1] = s_track[ind1] + abs(w*delta_x/old_mu)*Sigma_a #contribution to the initial cell
                ind1 = ind1 + 1
                while (ind1 != ind2):    #contribution to all cells between starting and end point
                    s_track[ind1] = s_track[ind1] + abs(w*1.0/old_mu)*Sigma_a
                    ind1 = ind1 + 1
                delta_x = new_x - float(ind1)
                if (new_x > 0.0 and new_x < 10.0):   #contribution to end cell
                    s_track[ind1] = s_track[ind1] + abs(w*delta_x/old_mu)*Sigma_a   
        if new_x < 0:  #particle leaked on the left
            #print('Leak to the left',i, new_x)
            break
        elif new_x > L:  #particle leaked on the right
            #print('Leak to the right',i, new_x)
            break
        else:
            ind = int(new_x)   #particle collided in cell ind
            s_collision[ind] = s_collision[ind] + w*Sigma_a/Sigma_t    #contribution to the collision estimator
            if random.random() < Sigma_s/Sigma_t:     #collision is scattering
                # Isotropic in CM
                cm_mu = random.random()*2-1           #isotropic scattering in COM
                # Convert to LAB
                lab_mu = (1+A*cm_mu)/np.sqrt(A**2+2*A*cm_mu+1)   #convert to LAB
                lab_phi = random.random()*2*3.1416               #sample uniform azimuthal angle
                new_mu = lab_mu*old_mu - np.sqrt(1-lab_mu**2)*np.sqrt(1-old_mu**2)*np.cos(lab_phi)   #rotate the vector
                old_mu = new_mu              
                old_x = new_x
            else:     #collision is absorption
                #print('Absorption Collision',i, new_x)
                ind = int(new_x)
                s_analog[ind] = s_analog[ind] + w    #contribution to the analog estimator
                break
    track_tally = track_tally + s_track                #once the history is the particle is over (outside the while loop)
    collision_tally = collision_tally + s_collision    #we add the history contribution to each tally
    analog_tally = analog_tally + s_analog
    track_tally_uq = track_tally_uq + s_track**2       #we also add the square of the history contribution that we will need
    collision_tally_uq = collision_tally_uq + s_collision**2    #to compute the variance
    analog_tally_uq = analog_tally_uq + s_analog**2
        

This code implements the track length over a superimposed mesh tally which makes it much more complicated.  We could have instead separates the slab into 10 cells and stopped particle at each boundary which would be have the tallying easier but with the tradeoff of searching for intersections and re-sampling distance travelled.

The other thing we can notice is how simple the collision and analog tallies are compared to the pathlength tallies.  The collision and absorption events are clearly defined while pathlenght requires computing the exact distance travelled at each flight.  This can also be seen when printing the tally accumulators. Analog and collision are integers in this case (since weight is always 1), while the pathlength are reals.

In [26]:
analog_tally

array([462., 625., 717., 802., 773., 804., 789., 739., 640., 468.])

In [27]:
collision_tally

array([466.2, 626.4, 744.2, 776.2, 796. , 802.8, 747.2, 719.6, 624.2,
       469.4])

In [28]:
track_tally

array([495.07754614, 628.97042249, 747.80925357, 774.51663612,
       801.77871323, 803.42181889, 771.26866466, 716.31579091,
       622.45815684, 453.66345229])

In [29]:
analog_tally_uq

array([462., 625., 717., 802., 773., 804., 789., 739., 640., 468.])

From the accumulators (history and history squared), we can now compute the mean and variance/std deviation.  The blocks below compute the sample mean and the sample standard deviation for all three tally types.

In [30]:
analog_tally = analog_tally/nps
analog_tally_uq = np.sqrt((analog_tally_uq/nps - analog_tally**2)/(nps-1))

In [31]:
collision_tally = collision_tally/nps
collision_tally_uq = np.sqrt((collision_tally_uq/nps - collision_tally**2)/(nps-1))

In [32]:
track_tally = track_tally/nps
track_tally_uq = np.sqrt((track_tally_uq/nps - track_tally**2)/(nps-1))

The means from all three tallies are equivalent, but the analog estimator has a larger uncertainty. If you lower the absorption xs of this problem, this will increase the difference between analog and collision/pathlength.  

The collision and pathlength estimators behave similarly for this case, but if you reduce the material density (lowering the macro xs), the pathlength estimator will performed better.

Most Monte Carlo codes will default to pathlength estimators (OpenMC, MCNP), but some tally types do not work with pathlenght and will thus use a collision or analog estimator.  This usually happens when post-collision information is needed (energy out filters and angle out filters).

SERPENT relies exclusively on collision estimators since their tracking is based on delta-tracking which would complicate the use of pathlength estimators.

In [33]:
print(analog_tally)
print(analog_tally_uq)
print(analog_tally_uq/analog_tally)

[0.0462 0.0625 0.0717 0.0802 0.0773 0.0804 0.0789 0.0739 0.064  0.0468]
[0.00209928 0.00242074 0.00258003 0.00271616 0.0026708  0.00271925
 0.00269596 0.00261621 0.00244765 0.00211221]
[0.04543907 0.03873177 0.03598374 0.03386735 0.03455113 0.03382152
 0.03416934 0.03540205 0.03824456 0.04513265]


In [34]:
print(collision_tally)
print(collision_tally_uq)
print(collision_tally_uq/collision_tally)

[0.04662 0.06264 0.07442 0.07762 0.0796  0.08028 0.07472 0.07196 0.06242
 0.04694]
[0.00142269 0.00164411 0.00187374 0.00185166 0.00191027 0.00192164
 0.00181274 0.00177293 0.00167996 0.00139351]
[0.03051683 0.02624695 0.02517797 0.02385549 0.02399842 0.02393666
 0.02426042 0.02463775 0.02691377 0.02968695]


In [35]:
print(track_tally)
print(track_tally_uq)
print(track_tally_uq/track_tally)

[0.04950775 0.06289704 0.07478093 0.07745166 0.08017787 0.08034218
 0.07712687 0.07163158 0.06224582 0.04536635]
[0.00147287 0.00142174 0.00160291 0.00161175 0.00166561 0.00168125
 0.00161998 0.0015521  0.00141987 0.00112711]
[0.02975023 0.02260418 0.02143478 0.02080977 0.02077388 0.02092616
 0.02100408 0.02166777 0.02281064 0.0248447 ]
